# Project of AI course (Sharif University of Technology): Phase 1

Setting up the environment

In [ ]:
%pip install pyrosm keplergl haversine pandas numpy

In [1]:
import datetime
import networkx as nx
import numpy as np
import pandas as pd
import pyrosm
import random
import ssl
import time

from keplergl import KeplerGl
from pyrosm import OSM, get_data

In [ ]:
# If you are using colab, you need to run this cell
from google.colab import output
output.enable_custom_widget_manager()

# Creating Pasdaran streets dataset (This part is not for students)

In [2]:
ssl._create_default_https_context = ssl._create_unverified_context
fp = get_data("tehran", directory="./")
print("Data was downloaded to:", fp)

Data was downloaded to: /Users/amirhossein/Documents/TA/AI/Project/Tehran.osm.pbf


<div dir="rtl">
سپس یک ابجکت osm
ایجاد می‌کنیم.
هدف این ابجکت خواندن دیتای ذخیره شده است.
دو ورودی می توان به آن داد که یکی آدرس فایل و یکی
bounding_box
جهت محدود کردن جغرافیایی دیتای خروجی است.
</br>
توسط get_network()
می‌توانیم نتورک خیابان‌های خروجی را فیلتر کنیم. بدین طریق که یک پارامتر network_type
داریم و به کمک آن نوع نتورک را مشخص می‌کنیم.
که در مثال زیر خیابان های قابل رانندگی انتخاب شده اند.
</div>

In [3]:
osm = pyrosm.OSM(fp)

nodes, edges = osm.get_network(nodes=True, network_type="driving")

<div dir="rtl">
ما  راس ها و یال‌هایی را خروجی می‌گیریم. راس ها درواقع تقاطع خیابان‌ها و یال‌ها خیابان اتصال کننده دو تقاطع اند.
</br>
حال هر کدام از این دو یک دیتافریم می‌باشند و می‌توانید از دستورات pandas برای آنها استفاده کنید.
</br>
همین طور که در head دیتافریم نمایش داده شده مشاهده می‌کنید هر راس یک طول و عرض جغرافیایی دارد که می‌توانیم با استفاده از آن محدوده‌های خاصی از نقشه را مشخص کنیم.
</div>

In [4]:
nodes.head()

,tags,lon,timestamp,version,lat,visible,changeset,id,geometry
0,None,51.347767,0,0,35.705349,False,0,1926828378,POINT (51.34777 35.70535)
1,None,51.346359,0,0,35.705315,False,0,873786679,POINT (51.34636 35.70531)
2,None,51.351189,0,0,35.707413,False,0,4607761177,POINT (51.35119 35.70741)
3,None,51.350956,0,0,35.707554,False,0,4607761176,POINT (51.35096 35.70755)
4,None,51.350838,0,0,35.707642,False,0,4607761175,POINT (51.35084 35.70764)


<div dir="rtl">
در ادامه با درنظر گرفتن نقاطی که عرض و طول جغرافیایی درون بازه خاصی دارند، رئوس محله پاسداران را از شهر مشخص می‌کنیم.
</div>

In [5]:
pasdaran_nodes = nodes[nodes['lat'] < 35.780]
pasdaran_nodes = pasdaran_nodes[pasdaran_nodes['lat'] > 35.762]
pasdaran_nodes = pasdaran_nodes[pasdaran_nodes['lon'] < 51.466]
pasdaran_nodes = pasdaran_nodes[pasdaran_nodes['lon'] > 51.440]
pasdaran_nodes = pasdaran_nodes.reset_index(drop=True)
pasdaran_nodes = pasdaran_nodes[['id', 'lat', 'lon']]
pasdaran_nodes

,id,lat,lon
0,423833948,35.772419,51.465652
1,423793584,35.771748,51.465084
2,177536927,35.776230,51.465034
3,177540293,35.771034,51.464462
4,423832815,35.772175,51.461494
...,...,...,...
1600,11158587867,35.763287,51.444275
1601,11158587869,35.763290,51.444252
1602,11158587868,35.763294,51.444233
1603,11194352478,35.764874,51.440830


<div dir="rtl">
حال برای دیتای خیابان‌ها صرفا id خیابان ، اینکه کدام دو راس را به هم اتصال می‌دهد، طول و یک طرفه بودنش را بدست می‌آوریم.
</div>

In [6]:
tehran_streets_pdf = edges[['id', 'u', 'v', 'length', 'oneway']]
tehran_streets_pdf

,id,u,v,length,oneway
0,4292665,1926828378,873786679,127.157,no
1,4292748,4607761177,4607761176,26.225,yes
2,4292748,4607761176,4607761175,14.463,yes
3,4292748,4607761175,9761557051,4.596,yes
4,4292748,9761557051,431492465,64.975,yes
...,...,...,...,...,...
293729,1222457613,11219542062,11219542060,7.341,yes
293730,1222457613,11219542060,11219542063,5.684,yes
293731,1222457613,11219542063,11219542061,4.578,yes
293732,1222457613,11219542061,339426026,11.576,yes


<div dir="rtl">
می‌توانیم با دادن آیدی یک خیابان مطابق زیر ویژگی های آن را اعم از یک طرفه بودن بررسی کنیم.
</div>

In [7]:
tehran_streets_pdf['oneway'][293236]

'yes'

<div dir="rtl">
از آنجا که ممکن است دیتاست اطلاعات کاملی از یک طرفه بودن ها نداشته باشد، بعضی از خانه ها None اند، برای رفع این موضوع این خانه ها را با no جایگزین می‌کنیم و فرض می‌کنیم دو طرفه اند.
</div>

In [8]:
tehran_streets_pdf = tehran_streets_pdf.fillna('no')

<div dir="rtl">
اینکه در دیتاست خیابان هایمان صرفا آیدی تقاطع ها را داریم به ما دید خوبی نمی‌دهد. بدین منظور این دیتافریم و دیتافریم مربوط به تقاطع های پاسداران را مرج می‌کنیم. برای جلوگیری از مشکل یکی بودن اسم ستون آیدی در این دو دیتافریم، آن را به street_id تغییر می‌دهیم.
</div>

In [9]:
tehran_streets_pdf = tehran_streets_pdf.rename(columns={'id': 'street_id'})
tehran_streets_pdf

,street_id,u,v,length,oneway
0,4292665,1926828378,873786679,127.157,no
1,4292748,4607761177,4607761176,26.225,yes
2,4292748,4607761176,4607761175,14.463,yes
3,4292748,4607761175,9761557051,4.596,yes
4,4292748,9761557051,431492465,64.975,yes
...,...,...,...,...,...
293729,1222457613,11219542062,11219542060,7.341,yes
293730,1222457613,11219542060,11219542063,5.684,yes
293731,1222457613,11219542063,11219542061,4.578,yes
293732,1222457613,11219542061,339426026,11.576,yes


<div dir="rtl">
دیتافریم ها را با یکدیگر مرج می‌کنیم تا برای هر پاره خط مختصات تقاطع های ابتدا و انتهایش را نیز داشته باشیم.
</div>

In [10]:
pasdaran_streets = tehran_streets_pdf.merge(pasdaran_nodes, left_on='u', right_on='id', how='inner')
pasdaran_streets = pasdaran_streets.rename(columns={'lat': 'u_lat', 'lon': 'u_lon'})

pasdaran_streets = pasdaran_streets.merge(pasdaran_nodes, left_on='v', right_on='id', how='inner')
pasdaran_streets = pasdaran_streets.rename(columns={'lat': 'v_lat', 'lon': 'v_lon'})
pasdaran_streets = pasdaran_streets.drop(['id_x', 'id_y'], axis=1)
pasdaran_streets

,street_id,u,v,length,oneway,u_lat,u_lon,v_lat,v_lon
0,5003747,423833948,423793584,90.570,no,35.772419,51.465652,35.771748,51.465084
1,17099043,177540293,423832815,296.273,yes,35.771034,51.464462,35.772175,51.461494
2,787945693,4429684144,423832815,62.573,no,35.772636,51.461891,35.772175,51.461494
3,687277053,177540293,423793585,97.898,no,35.771034,51.464462,35.770317,51.463833
4,36333153,9339973930,423793585,296.275,yes,35.771458,51.460865,35.770317,51.463833
...,...,...,...,...,...,...,...,...,...
1826,1203885252,11158587867,11158587869,2.108,yes,35.763287,51.444275,35.763290,51.444252
1827,1203885252,11158587869,11158587868,1.772,yes,35.763290,51.444252,35.763294,51.444233
1828,1208111703,4404038815,11194352478,15.794,no,35.764931,51.440990,35.764874,51.440830
1829,1208111704,11194352485,424065302,6.325,no,35.764084,51.441704,35.764072,51.441635


<div dir="rtl">
حال که با جزئیات ساختاری دیتاست آشنا شدیم، برای شهود بهتر نقاط انتخاب شده را روی نقشه نمایش می‌دهیم.
</br>
بدین منظور از کتابخانه folium بهره می‌بریم.
</div>

<div dir="rtl">
مپ را روی میانگین نقاط دیتاست پاسداران قرار می‌دهیم و سپس تمام شروع تقاطع‌ها را مارک می‌کنیم.
</div>

In [11]:
def visualize_streets_using_kepler(streets_pdf):
    assert 'u_lat' in streets_pdf.columns, "Your dataset must contain u_lat column"
    assert 'u_lon' in streets_pdf.columns, "Your dataset must contain u_lon column"

    streets_pdf = streets_pdf[['u_lat', 'u_lon']]

    kepler_map = KeplerGl(height=800, data={'all_data': streets_pdf})

    return kepler_map

kepler_map = visualize_streets_using_kepler(pasdaran_streets)
kepler_map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'all_data':           u_lat      u_lon
0     35.772419  51.465652
1     35.771034  51.464462
2 …

<div dir="rtl">
درنهایت دیتافریم های ساخته شده را در فایل های csv نظیر خود ذخیره می‌کنیم.
</div>

In [12]:
tehran_streets_pdf.to_csv('./tehran_streets.csv', index=False)
pasdaran_nodes.to_csv('./pasdaran_nodes.csv', index=False)
pasdaran_streets.to_csv('./pasdaran_streets.csv', index=False)

# Loading Pasdaran streets dataset

In [2]:
pasdaran_streets = pd.read_csv('./pasdaran_streets.csv')
pasdaran_nodes = pd.read_csv('./pasdaran_nodes.csv')

In [3]:
pasdaran_streets.head()

,street_id,u,v,length,oneway,u_lat,u_lon,v_lat,v_lon
0,5003747,423833948,423793584,90.570,no,35.772419,51.465652,35.771748,51.465084
1,17099043,177540293,423832815,296.273,yes,35.771034,51.464462,35.772175,51.461494
2,787945693,4429684144,423832815,62.573,no,35.772636,51.461891,35.772175,51.461494
3,687277053,177540293,423793585,97.898,no,35.771034,51.464462,35.770317,51.463833
4,36333153,9339973930,423793585,296.275,yes,35.771458,51.460865,35.770317,51.463833


In [4]:
pasdaran_nodes.head()

,id,lat,lon
0,423833948,35.772419,51.465652
1,423793584,35.771748,51.465084
2,177536927,35.776230,51.465034
3,177540293,35.771034,51.464462
4,423832815,35.772175,51.461494


In [5]:
def visualize_streets_using_kepler(streets_pdf):
    assert 'u_lat' in streets_pdf.columns, "Your dataset must contain u_lat column"
    assert 'u_lon' in streets_pdf.columns, "Your dataset must contain u_lon column"

    streets_pdf = streets_pdf[['u_lat', 'u_lon']]

    kepler_map = KeplerGl(height=800, data={'all_data': streets_pdf})

    return kepler_map

kepler_map = visualize_streets_using_kepler(pasdaran_streets)
kepler_map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'all_data':           u_lat      u_lon
0     35.772419  51.465652
1     35.771034  51.464462
2 …

# Creating dataset for trip

As you understood, until now, we loaded 2 datasets, `pasdaran_nodes` and `pasdaran_streets`. Now, we should create a dataset from these, such that we can use it for our trip. Each item in this dataset is called a `ride`. We will explain more as we go on.

First, we should create a **trip** for a driver. For this sake, we should choose 2 random points in our map as the source and destination of the trip. Now, we should find the streets that connect these 2 points. Each street has a length. You should find the shortest path between these 2 points (source and destination) which are in the `pasdaran_streets`. Note that this path should be the **shortest path** meaning that the sum of the lengths of streets should be minimum.

In [6]:
def check_node_in_street_df(u,streets):
  return u in streets["u"].values or u in streets["v"].values

In [7]:
import numpy as np
import networkx as nx


def sample_trip(nodes, streets):
    # sample source and destination nodes
    source_node = np.random.choice(nodes["id"].values)
    destination_node = np.random.choice(nodes["id"].values)
    prestreets = streets
    # find the shortest path between source and destination
    streets = nx.from_pandas_edgelist(
        streets, "u", "v", ["street_id", "length", "oneway"]
    )
    while not check_node_in_street_df(source_node,prestreets) or not check_node_in_street_df(destination_node,prestreets) or not nx.has_path(streets,source=source_node,target=destination_node):
      source_node = np.random.choice(nodes["id"].values)
      destination_node = np.random.choice(nodes["id"].values)

    shortest_path = nx.shortest_path(
            streets, source=source_node, target=destination_node, weight="length"
        )
    return shortest_path, source_node, destination_node


sampled_trip, sampled_source, sampled_dest = sample_trip(
    pasdaran_nodes, pasdaran_streets
)

In [8]:
sampled_trip[:5] # contains the street_ids which create the shortest path

[4695975717, 1721683025, 423815697, 423815691, 423815693]

Visualize the sampled trip (in the previous cell) on the map. Use `kepler` library for this purpose.

In [9]:
def visualize_streets_using_kepler(sampled_trip, sampled_source, sampled_dest,
                                    pasdaran_nodes_pdf=pasdaran_nodes,
                                    pasdaran_streets_pdf=pasdaran_streets):
    data = []
    kepler_config = {'version': 'v1', 'config': {'visState': {'filters': [{'dataId': ['sample_ride'], 'id': 'io3gbsqzf', 'name': ['timestamp'], 'type': 'timeRange', 'value': [1700239623660, 1700239797294.5928], 'enlarged': True, 'plotType': 'histogram', 'animationWindow': 'incremental', 'yAxis': None, 'speed': 1}], 'layers': [{'id': 'ayoewf8', 'type': 'point', 'config': {'dataId': 'sample_ride', 'label': 'Rides', 'color': [18, 147, 154], 'highlightColor': [252, 242, 26, 255], 'columns': {'lat': 'lat', 'lng': 'lng', 'altitude': None}, 'isVisible': True, 'visConfig': {'radius': 10, 'fixedRadius': False, 'opacity': 0.8, 'outline': False, 'thickness': 2, 'strokeColor': None, 'colorRange': {'name': 'ColorBrewer RdYlGn-6', 'type': 'diverging', 'category': 'ColorBrewer', 'colors': ['#1a9850', '#91cf60', '#d9ef8b', '#fee08b', '#fc8d59', '#d73027'], 'reversed': True}, 'strokeColorRange': {'name': 'Global Warming', 'type': 'sequential', 'category': 'Uber', 'colors': ['#5A1846', '#900C3F', '#C70039', '#E3611C', '#F1920E', '#FFC300']}, 'radiusRange': [0, 42.9], 'filled': True}, 'hidden': False, 'textLabel': []}, 'visualChannels': {'colorField': {'name': 'timestamp', 'type': 'timestamp'}, 'colorScale': 'quantile', 'strokeColorField': None, 'strokeColorScale': 'quantile', 'sizeField': None, 'sizeScale': 'linear'}}], 'interactionConfig': {'tooltip': {'fieldsToShow': {'sample_ride': [{'name': 'timestamp', 'format': None}, {'name': 'type', 'format': None}]}, 'compareMode': False, 'compareType': 'absolute', 'enabled': True}, 'brush': {'size': 0.5, 'enabled': False}, 'geocoder': {'enabled': False}, 'coordinate': {'enabled': False}}, 'layerBlending': 'normal', 'splitMaps': [], 'animationConfig': {'currentTime': None, 'speed': 1}}, 'mapState': {'bearing': 0, 'dragRotate': False, 'latitude': 35.757444215062094, 'longitude': 51.457852144642786, 'pitch': 0, 'zoom': 12.66606473277238, 'isSplit': False}, 'mapStyle': {'styleType': 'dark', 'topLayerGroups': {}, 'visibleLayerGroups': {'label': True, 'road': True, 'border': False, 'building': True, 'water': True, 'land': True, '3d building': False}, 'threeDBuildingColor': [9.665468314072013, 17.18305478057247, 31.1442867897876], 'mapStyles': {}}}}

    sample_current_time = time.time()
    for _id in sampled_trip:
        random_interval_time = random.gauss(5, 1)
        _type = None
        if _id == sampled_source:
            _type = 'source'
        elif _id == sampled_dest:
            _type = 'target'
        else:
            _type = 'in_ride'

        data.append({
            'lat': pasdaran_nodes_pdf[pasdaran_nodes_pdf["id"] == _id]["lat"].values[0],
            'lng': pasdaran_nodes_pdf[pasdaran_nodes_pdf["id"] == _id]["lon"].values[0],
            'timestamp': datetime.datetime.fromtimestamp(sample_current_time),
            'type': _type,
        })
        sample_current_time += random_interval_time


    data_pdf = pd.DataFrame(data)

    kepler_map = KeplerGl(height=800, data={'sample_ride': data_pdf}, config=kepler_config)

    return kepler_map

kepler_map = visualize_streets_using_kepler(sampled_trip, sampled_source, sampled_dest)
kepler_map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [{'dataId': ['sample_ride'], 'id': 'io3gb…

Now, we are going to get the points in the middle of the streets. Before, we only had the intersections of the streets(start and end nodes of a section). But we know in real life, we get gps locations also in between and not necessarily in the intersections. So, we should add these points to our data. We call every entity of this type as `Ride`. A ride is defined by 3 parameters:
1. `ride_speed`: The speed of the ride which driver is driving with. We assume that the driver is driving with a constant speed.
2. `driver_gps_time`: The time interval that the driver sends his/her gps location to the server. (e.g. every 5 seconds)
3. `trip`: The specific trip that the driver is driving in. (we created this in the previous cells)

Now, you should define a function that gets these parameters and returns the locations (lat/longs) in the ride regarding the given parameters.

In [10]:
def visualize_a_ride(ride_speed, ride_gps_time, trip):
    ride_locations = []
    for i in range(len(trip) - 1):
        u = pasdaran_nodes[pasdaran_nodes["id"] == trip[i]]
        v = pasdaran_nodes[pasdaran_nodes["id"] == trip[i + 1]]
        # get locations between u and v, wrt ride_speed and ride_gps_time
        u_lat = u["lat"].values[0]
        u_lon = u["lon"].values[0]
        v_lat = v["lat"].values[0]
        v_lon = v["lon"].values[0]
        # get the distance between u and v
        distance = np.sqrt((u_lat - v_lat) ** 2 + (u_lon - v_lon) ** 2)
        # get the time between u and v
        time = distance / ride_speed
        # get the number of locations between u and v
        num_locations = int(time / ride_gps_time)
        # get the locations between u and v
        for j in range(num_locations):
            ride_locations.append(
                [
                    u_lat + (v_lat - u_lat) * j / num_locations,
                    u_lon + (v_lon - u_lon) * j / num_locations,
                ]
            )

    return ride_locations

See a sample ride, printed out.

In [11]:
visualized_ride = visualize_a_ride(ride_speed=1e-3, ride_gps_time=0.45, trip=sampled_trip)
visualized_ride

[[35.77680206298828, 51.45387649536133],
 [35.777008056640625, 51.453216552734375],
 [35.77714920043945, 51.452728271484375],
 [35.77729034423828, 51.452239990234375],
 [35.77743148803711, 51.451751708984375],
 [35.77757263183594, 51.451263427734375],
 [35.777716636657715, 51.45078182220459],
 [35.77786064147949, 51.450300216674805],
 [35.77800464630127, 51.44981861114502],
 [35.77814865112305, 51.449337005615234],
 [35.778865814208984, 51.44964599609375],
 [35.77895991007487, 51.449170430501304],
 [35.77905400594076, 51.44869486490885]]

Visualize the sample ride on the map using `kepler` library. You should define an animation based on the order of the points in the ride, so you can play the animation in the visualization and see the ride!

In [12]:
def visualize_ride_using_kepler(ride):
    data = []
    kepler_config = {'version': 'v1', 'config': {'visState': {'filters': [{'dataId': ['sample_ride'], 'id': 'io3gbsqzf', 'name': ['timestamp'], 'type': 'timeRange', 'value': [1700239623660, 1700239797294.5928], 'enlarged': True, 'plotType': 'histogram', 'animationWindow': 'incremental', 'yAxis': None, 'speed': 1}], 'layers': [{'id': 'ayoewf8', 'type': 'point', 'config': {'dataId': 'sample_ride', 'label': 'Rides', 'color': [18, 147, 154], 'highlightColor': [252, 242, 26, 255], 'columns': {'lat': 'lat', 'lng': 'lng', 'altitude': None}, 'isVisible': True, 'visConfig': {'radius': 10, 'fixedRadius': False, 'opacity': 0.8, 'outline': False, 'thickness': 2, 'strokeColor': None, 'colorRange': {'name': 'ColorBrewer RdYlGn-6', 'type': 'diverging', 'category': 'ColorBrewer', 'colors': ['#1a9850', '#91cf60', '#d9ef8b', '#fee08b', '#fc8d59', '#d73027'], 'reversed': True}, 'strokeColorRange': {'name': 'Global Warming', 'type': 'sequential', 'category': 'Uber', 'colors': ['#5A1846', '#900C3F', '#C70039', '#E3611C', '#F1920E', '#FFC300']}, 'radiusRange': [0, 42.9], 'filled': True}, 'hidden': False, 'textLabel': []}, 'visualChannels': {'colorField': {'name': 'timestamp', 'type': 'timestamp'}, 'colorScale': 'quantile', 'strokeColorField': None, 'strokeColorScale': 'quantile', 'sizeField': None, 'sizeScale': 'linear'}}], 'interactionConfig': {'tooltip': {'fieldsToShow': {'sample_ride': [{'name': 'timestamp', 'format': None}, {'name': 'type', 'format': None}]}, 'compareMode': False, 'compareType': 'absolute', 'enabled': True}, 'brush': {'size': 0.5, 'enabled': False}, 'geocoder': {'enabled': False}, 'coordinate': {'enabled': False}}, 'layerBlending': 'normal', 'splitMaps': [], 'animationConfig': {'currentTime': None, 'speed': 1}}, 'mapState': {'bearing': 0, 'dragRotate': False, 'latitude': 35.757444215062094, 'longitude': 51.457852144642786, 'pitch': 0, 'zoom': 12.66606473277238, 'isSplit': False}, 'mapStyle': {'styleType': 'dark', 'topLayerGroups': {}, 'visibleLayerGroups': {'label': True, 'road': True, 'border': False, 'building': True, 'water': True, 'land': True, '3d building': False}, 'threeDBuildingColor': [9.665468314072013, 17.18305478057247, 31.1442867897876], 'mapStyles': {}}}}

    sample_current_time = time.time()
    for i, item in enumerate(ride):
        random_interval_time = random.gauss(5, 1)
        _type = "in_ride" if i != 0 and i != len(ride) - 1 else "source" if i == 0 else "target"

        data.append({
            'lat': item[0],
            'lng': item[1],
            'timestamp': datetime.datetime.fromtimestamp(sample_current_time),
            'type': _type,
        })
        sample_current_time += random_interval_time


    data_pdf = pd.DataFrame(data)

    kepler_map = KeplerGl(height=800, data={'sample_ride': data_pdf}, config=kepler_config)

    return kepler_map

kepler_map = visualize_ride_using_kepler(visualized_ride)
kepler_map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [{'dataId': ['sample_ride'], 'id': 'io3gb…

Now, to complete our dataset, we need to put some random noise on the gps locations. For this sake, we easily add some random noise to the lat/longs of a created `ride`. You should define a function that gets a `ride` and adds some random noise to the lat/longs of the ride. Your function should take in 2 parameters: _ride_ and _gps\_noise\_radius_ which is the radius of the circle that you want to add noise to the gps locations. (e.g. 10 meters). The gps noise in assumed to be a normal distribution.

In [13]:
# put a random noise on each (lat, lon) location to simulate the GPS noise (suppoes GPS radius is 3 - 8 meters)
def add_noise_to_ride(ride, gps_noise_radius):
    noisy_ride = []
    for location in ride:
        noise = np.random.normal(0, gps_noise_radius)
        noisy_ride.append(
            [
                location[0] + noise,
                location[1] + noise,
            ]
        )

    return noisy_ride


noisy_ride = add_noise_to_ride(visualized_ride, gps_noise_radius=1e-4)
noisy_ride

[[35.77684092689143, 51.45391535926448],
 [35.77705190825601, 51.45326040434976],
 [35.77715978561018, 51.4527388566551],
 [35.77730755957582, 51.45225720557191],
 [35.77737337238108, 51.45169359332834],
 [35.777551615879126, 51.451242411777564],
 [35.777569063017495, 51.45063424856437],
 [35.77798417116273, 51.45042374635804],
 [35.77797630019279, 51.44979026503654],
 [35.77806431956048, 51.449252674052666],
 [35.778836265823145, 51.44961644770791],
 [35.77896128224182, 51.44917180266826],
 [35.77900067050033, 51.448641529468425]]

Visualize a sample `ride` and its corresponding `noisy_ride` on the map.

In [14]:
def visualize_ride_using_kepler(ride, noisy_ride):
    data = []
    kepler_config = {'version': 'v1', 'config': {'visState': {'filters': [{'dataId': ['sample_ride'], 'id': 'io3gbsqzf', 'name': ['timestamp'], 'type': 'timeRange', 'value': [1700239623660, 1700239797294.5928], 'enlarged': True, 'plotType': 'histogram', 'animationWindow': 'incremental', 'yAxis': None, 'speed': 1}], 'layers': [{'id': 'ayoewf8', 'type': 'point', 'config': {'dataId': 'sample_ride', 'label': 'Rides', 'highlightColor': [252, 242, 26, 255], 'columns': {'lat': 'lat', 'lng': 'lng', 'altitude': None}, 'isVisible': True, 'visConfig': {'radius': 10, 'fixedRadius': False, 'opacity': 0.8, 'outline': False, 'thickness': 2, 'strokeColor': None, 'colorRange': {'name': 'ColorBrewer RdYlGn-6', 'type': 'diverging', 'category': 'ColorBrewer', 'colors': ['#1a9850', '#91cf60', '#d9ef8b', '#fee08b', '#fc8d59', '#d73027'], 'reversed': True}, 'strokeColorRange': {'name': 'Global Warming', 'type': 'sequential', 'category': 'Uber', 'colors': ['#5A1846', '#900C3F', '#C70039', '#E3611C', '#F1920E', '#FFC300']}, 'radiusRange': [0, 42.9], 'filled': True}, 'hidden': False, 'textLabel': []}, 'visualChannels': {'colorField': {'name': 'timestamp', 'type': 'timestamp'}, 'colorScale': 'quantile', 'strokeColorField': None, 'strokeColorScale': 'quantile', 'sizeField': None, 'sizeScale': 'linear'}}], 'interactionConfig': {'tooltip': {'fieldsToShow': {'sample_ride': [{'name': 'timestamp', 'format': None}, {'name': 'type', 'format': None}]}, 'compareMode': False, 'compareType': 'absolute', 'enabled': True}, 'brush': {'size': 0.5, 'enabled': False}, 'geocoder': {'enabled': False}, 'coordinate': {'enabled': False}}, 'layerBlending': 'normal', 'splitMaps': [], 'animationConfig': {'currentTime': None, 'speed': 1}}, 'mapState': {'bearing': 0, 'dragRotate': False, 'latitude': 35.757444215062094, 'longitude': 51.457852144642786, 'pitch': 0, 'zoom': 12.66606473277238, 'isSplit': False}, 'mapStyle': {'styleType': 'dark', 'topLayerGroups': {}, 'visibleLayerGroups': {'label': True, 'road': True, 'border': False, 'building': True, 'water': True, 'land': True, '3d building': False}, 'threeDBuildingColor': [9.665468314072013, 17.18305478057247, 31.1442867897876], 'mapStyles': {}}}}

    sample_current_time = time.time()
    # add "ride" points as green and "noisy_ride" points as red
    for i, item in enumerate(ride):
        random_interval_time = random.gauss(5, 1)
        _type = "in_ride" if i != 0 and i != len(ride) - 1 else "source" if i == 0 else "target"

        data.append({
            'lat': item[0],
            'lng': item[1],
            'timestamp': datetime.datetime.fromtimestamp(sample_current_time),
            'type': _type,
        })
        sample_current_time += random_interval_time
    
    for i, item in enumerate(noisy_ride):
        random_interval_time = random.gauss(5, 1)
        _type = "in_ride" if i != 0 and i != len(noisy_ride) - 1 else "source" if i == 0 else "target"

        data.append({
            'lat': item[0],
            'lng': item[1],
            'timestamp': datetime.datetime.fromtimestamp(sample_current_time),
            'type': _type,
        })
        sample_current_time += random_interval_time

    # put main points as green and noisy points as red
    data_pdf = pd.DataFrame(data)

    kepler_map = KeplerGl(height=800, data={'sample_ride': data_pdf}, config=kepler_config)

    return kepler_map

# Assuming visualized_ride and noisy_ride are defined elsewhere in your code
visualize_ride_using_kepler(visualized_ride, noisy_ride)


User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [{'dataId': ['sample_ride'], 'id': 'io3gb…

Now, we create our dataset, We create 20 noisy_rides for each trip.

In [15]:
def create_trip_dataset(trips = 20):
    df = pd.DataFrame(columns=['trip_id', 'index', 'source' ,'destination', 'lat', 'lon', 'ride_speed', 'ride_gps_time'])
    df_orig = pd.DataFrame(columns=['trip_id', 'index', 'source' ,'destination', 'lat', 'lon', 'ride_speed', 'ride_gps_time'])
    for i in range(trips):
        rs = np.round(np.random.normal(1e-3, 1e-4),6)
        rgt = np.round(np.random.normal(0.45, 0.1),3)
        sampled_trip, sampled_source, sampled_dest = sample_trip(pasdaran_nodes, pasdaran_streets)
        visualized_ride = visualize_a_ride(ride_speed=rs, ride_gps_time=rgt, trip=sampled_trip)
        noisy_ride = add_noise_to_ride(visualized_ride, gps_noise_radius=1e-4)
        for j in range(len(noisy_ride)):
            df = pd.concat([df,pd.DataFrame({'trip_id': i, 'index': j, 'source': sampled_source, 'destination': sampled_dest, 'lat': [noisy_ride[j][0]], 'lon': [noisy_ride[j][1]], 'ride_speed': rs, 'ride_gps_time':rgt})])
            df_orig = pd.concat([df_orig,pd.DataFrame({'trip_id': i, 'index': j, 'source': sampled_source, 'destination': sampled_dest, 'lat': [visualized_ride[j][0]], 'lon': [visualized_ride[j][1]], 'ride_speed': rs, 'ride_gps_time':rgt})])
    return df, df_orig

trip_dataset_1, original_dt = create_trip_dataset()

/var/folders/rd/pptsn4g94yn355rms_txgp680000gn/T/ipykernel_2825/3250589871.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df,pd.DataFrame({'trip_id': i, 'index': j, 'source': sampled_source, 'destination': sampled_dest, 'lat': [noisy_ride[j][0]], 'lon': [noisy_ride[j][1]], 'ride_speed': rs, 'ride_gps_time':rgt})])
/var/folders/rd/pptsn4g94yn355rms_txgp680000gn/T/ipykernel_2825/3250589871.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_orig = pd.concat([df_orig,pd.Da

In [16]:
trip_dataset_1.to_csv("noisy_test_dataset.csv")
original_dt.to_csv("original_test_dataset.csv")

# HMM

In this section, we want to use the dataset that we created in the previous section to train an HMM model. We want to use HMM models to  **denoise** the gps locations of the rides. We assume that the gps locations of the rides are noisy and we want to denoise them using HMM models. The observations in each _timestamp_ are the observed noisy points in the dataset.

You should implement a class (or function) named HMM which gets the noisy observations and the `streets` of the map (of course, this is not noisy. we saw that as `pasdaran_streets` in the previous section). You should calculate the transition matrix and emission matrix of the HMM model. You should also implement the `viterbi` algorithm to find the most probable sequence of hidden states (which are the points in the streets) given the noisy observations. Finally, your implementation should be capable of returning the denoised gps locations of the rides. Note that how you define your features, and your states. Be creative!

In [17]:
import math

def distance_point_line(start, end, pnt):
    def dot(v,w):
        x,y = v
        X,Y = w
        return x*X + y*Y

    def length(v):
        x,y = v
        return math.sqrt(x*x + y*y)

    def vector(b,e):
        x,y = b
        X,Y = e
        return (X-x, Y-y)

    def unit(v):
        x,y = v
        mag = length(v)
        return (x/mag, y/mag)

    def distance(p0,p1):
        return length(vector(p0,p1))

    def scale(v,sc):
        x,y = v
        return (x * sc, y * sc)

    def add(v,w):
        x,y = v
        X,Y = w
        return (x+X, y+Y)
    line_vec = vector(start, end)
    pnt_vec = vector(start, pnt)
    line_len = length(line_vec)
    line_unitvec = unit(line_vec)
    pnt_vec_scaled = scale(pnt_vec, 1.0/line_len)
    t = dot(line_unitvec, pnt_vec_scaled)
    if t < 0.0:
        t = 0.0
    elif t > 1.0:
        t = 1.0
    nearest = scale(line_vec, t)
    dist = distance(nearest, pnt_vec)
    nearest = add(nearest, start)
    return (dist, nearest)

print(distance_point_line([1, 1], [3, 3], [4, 4]))


(1.4142135623730951, (3.0, 3.0))


In [18]:
def shortest_dist(source_node, destination_node, nodes, streets):
    # sample source and destination nodes
    prestreets = streets
    # find the shortest path between source and destination
    streets = nx.from_pandas_edgelist(
        streets, "u", "v", ["street_id", "length", "oneway"]
    )
    #while not check_node_in_street_df(source_node,prestreets) or not check_node_in_street_df(destination_node,prestreets) or not nx.has_path(streets,source=source_node,target=destination_node):
      #source_node = np.random.choice(nodes["id"].values)
      #destination_node = np.random.choice(nodes["id"].values)
    try:
        shortest_path_length = nx.shortest_path_length(
                streets, source=source_node, target=destination_node, weight="length"
            )
        return shortest_path_length
    except Exception as e:
        return 100000
print(shortest_dist(423833948, 2022634168, pasdaran_nodes, pasdaran_streets))


100000


In [19]:
import scipy.stats
from itertools import repeat

def prob_point_given_line(start, end, point):
    return scipy.stats.norm(0, 1e-4).pdf(distance_point_line(start, end, point)[0])


def HMM(observations, streets):
    nSamples = len(observations)
    nStates = len(streets)
    probs = [[] for i in repeat(None, nSamples)]
    #print(probs)
    prob_streets = [[] for i in repeat(None, nSamples)]
    prv_streets = [[] for i in repeat(None, nSamples - 1)]


    obs_0 = [observations.iloc[0]["lat"], observations.iloc[0]["lon"]]
    for i, street in enumerate(streets.iterrows()):
      street = street[1]
      prob_of_select_first = prob_point_given_line([street["u_lat"], street["u_lon"]], [street["v_lat"], street["v_lon"]], obs_0)
      if prob_of_select_first > 1e-5:
          probs[0].append(prob_of_select_first)
          prob_streets[0].append(street)
    #print(prob_streets[0])

    for j in range(1, nSamples):
        #print("sample" + str(j))
        for i, street in enumerate(streets.iterrows()):

            street = street[1]
            #state_probs = []
            obs_prev = [observations.iloc[j-1]["lat"], observations.iloc[j-1]["lon"]]
            obs_present = [observations.iloc[j]["lat"], observations.iloc[j]["lon"]]

            dist_obs = np.linalg.norm(np.array(obs_prev) - np.array(obs_present))
            street_1_start, street_1_end = street["u"], street["v"]
            all_trans = []
            all_prev_streets = []
            emit = prob_point_given_line([street["u_lat"], street["u_lon"]], [street["v_lat"], street["v_lon"]], obs_present)
            if emit > 1e-5:
                for k, street2 in enumerate(prob_streets[j - 1]):
                    #street2 = street2[1]

                    street_2_start, street_2_end = street2["u"], street2["v"]

                    proj_point1 = (distance_point_line([street["u_lat"], street["u_lon"]], [street["v_lat"], street["v_lon"]], obs_present))
                    proj_point2 = (distance_point_line([street2["u_lat"], street2["u_lon"]], [street2["v_lat"], street2["v_lon"]], obs_prev))


                    state_to_start1 = (street["u_lat"] - proj_point1[1][0]) / (street["u_lat"] - street["v_lat"] + 1e-6) * street["length"]
                    state_to_start2 = (street2["u_lat"] - proj_point2[1][0]) / (street2["u_lat"] - street2["v_lat"] + 1e-6) * street2["length"]
                    if street2["street_id"] != street["street_id"]:
                        d1 = shortest_dist(street["u"], street2["u"], pasdaran_nodes, pasdaran_streets) #+ state_to_start1 + state_to_start2
                        d2 = shortest_dist(street["u"], street2["v"], pasdaran_nodes, pasdaran_streets) #+ state_to_start1 + street2["length"] - state_to_start2
                        d3 = shortest_dist(street["v"], street2["u"], pasdaran_nodes, pasdaran_streets) #+ street["length"] - state_to_start1 + state_to_start2
                        d4 = shortest_dist(street["v"], street2["v"], pasdaran_nodes, pasdaran_streets) #+ street["length"] + street2["length"] - state_to_start1 + state_to_start2
                        trans = abs(dist_obs * street["length"] / (abs(street["u_lat"] - street2["u_lat"])) - min(d1, d2, d3, d4))
                    else:
                        trans = abs(state_to_start1 - state_to_start2)
                #trans = abs(dist_obs * street["length"] / abs(street["u_lat"] - street2["u_lat"]) - min(d1, d2, d3, d4))

                    trans = np.exp(-trans / 100) / 100
                    #print(trans)
                    #print(trans, emit, probs[j - 1][k])
                    if trans * emit * probs[j - 1][k] > 1e-10:
                      all_trans.append(trans * emit * probs[j - 1][k])
                      all_prev_streets.append(street2)
                if len(all_trans) > 0:
                  max_prob = max(all_trans)
                  #print(max_prob)
                #print(probs[j])
                  probs[j].append(max_prob)
                  prob_streets[j].append(street)
                  prv_streets[j-1].append(all_prev_streets[all_trans.index(max_prob)])

    best_path = []
    best_path.append(prob_streets[-1][probs[-1].index(max(probs[-1]))])
    for j in range(nSamples - 1, 0, -1):
        index = probs[j].index(max(probs[j]))
        best_path.append(prv_streets[j-1][index])

    final_project = []

    for j in range(0, nSamples):
        obs = [observations.iloc[nSamples - j - 1]["lat"], observations.iloc[nSamples - j - 1]["lon"]]
        final_project.append(distance_point_line([best_path[j]["u_lat"], best_path[j]["u_lon"]], [best_path[j]["v_lat"], best_path[j]["v_lon"]], obs))

    return [list(i[1]) for i in reversed(final_project)]

first_result = HMM(trip_dataset_1.loc[trip_dataset_1['trip_id'] == 0], pasdaran_streets)

/var/folders/rd/pptsn4g94yn355rms_txgp680000gn/T/ipykernel_2825/3847590859.py:57: RuntimeWarning: divide by zero encountered in scalar divide
  trans = abs(dist_obs * street["length"] / (abs(street["u_lat"] - street2["u_lat"])) - min(d1, d2, d3, d4))


Visualize a sample denoised _ride_ using your HMM model. Put `original_points` and `denoised_points` on the map. Use different colors for each of them to make them distinct from eachother. You should use `kepler` library for this purpose.

In [20]:
def visualize_result_of_denoising(denoised_points, original_points):
    data = []
    kepler_config = {
        "version": "v1",
        "config": {
            "visState": {
                "layers": [
                    {
                        "id": "denoised_layer",
                        "type": "point",
                        "config": {
                            "dataId": "denoised_data",
                            "label": "Denoised Points",
                            "columns": {"lat": "lat", "lng": "lng", "altitude": None},
                            "isVisible": True,
                            "visConfig": {
                                "radius": 10,
                                "fixedRadius": False,
                                "opacity": 0.8,
                                "outline": False,
                                "thickness": 2,
                                "strokeColor": None,
                                "color": [0, 255, 0],  # Green for 'denoised'
                                "filled": True,
                            },
                            "hidden": False,
                            "textLabel": [],
                        },
                        "visualChannels": {
                            "colorField": {"name": "timestamp", "type": "timestamp"},
                            "colorScale": "quantile",
                            "strokeColorField": None,
                            "strokeColorScale": "quantile",
                            "sizeField": None,
                            "sizeScale": "linear",
                        },
                    },
                    {
                        "id": "original_layer",
                        "type": "point",
                        "config": {
                            "dataId": "original_data",
                            "label": "Original Points",
                            "columns": {"lat": "lat", "lng": "lng", "altitude": None},
                            "isVisible": True,
                            "visConfig": {
                                "radius": 10,
                                "fixedRadius": False,
                                "opacity": 0.8,
                                "outline": False,
                                "thickness": 2,
                                "strokeColor": None,
                                "color": [255, 0, 0],  # Red for 'original'
                                "filled": True,
                            },
                            "hidden": False,
                            "textLabel": [],
                        },
                        "visualChannels": {
                            "colorField": {"name": "timestamp", "type": "timestamp"},
                            "colorScale": "quantile",
                            "strokeColorField": None,
                            "strokeColorScale": "quantile",
                            "sizeField": None,
                            "sizeScale": "linear",
                        },
                    },
                ],
                "interactionConfig": {
                    "tooltip": {
                        "fieldsToShow": {
                            "sample_ride": [
                                {"name": "timestamp", "format": None},
                                {"name": "type", "format": None},
                            ]
                        },
                        "compareMode": False,
                        "compareType": "absolute",
                        "enabled": True,
                    },
                    "brush": {"size": 0.5, "enabled": False},
                    "geocoder": {"enabled": False},
                    "coordinate": {"enabled": False},
                },
                "layerBlending": "normal",
                "splitMaps": [],
                "animationConfig": {"currentTime": None, "speed": 1},
            },
            "mapState": {
                "bearing": 0,
                "dragRotate": False,
                "latitude": 35.757444215062094,
                "longitude": 51.457852144642786,
                "pitch": 0,
                "zoom": 12.66606473277238,
                "isSplit": False,
            },
            "mapStyle": {
                "styleType": "dark",
                "topLayerGroups": {},
                "visibleLayerGroups": {
                    "label": True,
                    "road": True,
                    "border": False,
                    "building": True,
                    "water": True,
                    "land": True,
                    "3d building": False,
                },
                "threeDBuildingColor": [
                    9.665468314072013,
                    17.18305478057247,
                    31.1442867897876,
                ],
                "mapStyles": {},
            },
        },
    }

    # add "denoised" points as green
    for i, item in enumerate(denoised_points):
        _type = (
            "in_ride"
            if i != 0 and i != len(denoised_points) - 1
            else "source"
            if i == 0
            else "target"
        )

        data.append(
            {
                "lat": item[0],
                "lng": item[1],
                "type": _type,
            }
        )

    # put denoised points as green
    denoised_data_pdf = pd.DataFrame(data)

    data = []  # Reset data for original points

    # add "original" points as red
    for i, item in enumerate(original_points.iterrows()):
        _type = (
            "in_ride"
            if i != 0 and i != len(original_points) - 1
            else "source"
            if i == 0
            else "target"
        )

        data.append(
            {
                "lat": item[1]["lat"],
                "lng": item[1]["lon"],
                "type": _type,
            }
        )

    # put original points as red
    original_data_pdf = pd.DataFrame(data)

    # Combine both data frames
    data_frames = {
        "denoised_data": denoised_data_pdf,
        "original_data": original_data_pdf,
    }

    kepler_map = KeplerGl(height=800, data=data_frames, config=kepler_config)

    return kepler_map


# Assuming first_result and trip_dataset_1 are defined elsewhere in your code
visualize_result_of_denoising(
    first_result, trip_dataset_1.loc[trip_dataset_1["trip_id"] == 0]
)

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'layers': [{'id': 'denoised_layer', 'type': 'point',…

# Testing your model

In this section, you should test your model. You could use your dataset (which you generated, step-by-step) until now, to implement, debug and train your model. Now, you should load a `test_dataset` that we provided to you as `test_dataset.csv` and run your model on it to denoise the points. You should put the denoised points in a csv file named `denoised_test_dataset.csv` and submit it with your notebook. Also, You should report the **Mean haversine distance (MHD)** as the measaure of the loss of your model. Go through each trip in the `test_dataset` and calculate the MHD for each trip. Finally, report the average of the MHDs of all the trips in the `test_dataset` as the MHD of your model.

In [21]:
from haversine import haversine
from tqdm import tqdm


def mean_haversine_distance(denoised_points, original_points):
    distances = []
    for i, item in enumerate(denoised_points):
        distances.append(
            haversine(
                (item[0], item[1]),
                (original_points.iloc[i]["lat"], original_points.iloc[i]["lon"]),
            )
        )

    return np.mean(distances)


test_dataset_noisy = pd.read_csv("noisy_test_dataset.csv")
test_dataset_original = pd.read_csv("original_test_dataset.csv")

test_dataset_original.head()

,Unnamed: 0,trip_id,index,source,destination,lat,lon,ride_speed,ride_gps_time
0,0,0,0,4695978684,4698502266,35.778339,51.454990,0.001095,0.507
1,0,0,1,4695978684,4698502266,35.778603,51.453945,0.001095,0.507
2,0,0,2,4695978684,4698502266,35.778553,51.453321,0.001095,0.507
3,0,0,3,4695978684,4698502266,35.778503,51.452698,0.001095,0.507
4,0,0,4,4695978684,4698502266,35.778378,51.451622,0.001095,0.507


In [22]:
test_dataset_noisy.head()

,Unnamed: 0,trip_id,index,source,destination,lat,lon,ride_speed,ride_gps_time
0,0,0,0,4695978684,4698502266,35.778304,51.454955,0.001095,0.507
1,0,0,1,4695978684,4698502266,35.778631,51.453973,0.001095,0.507
2,0,0,2,4695978684,4698502266,35.778324,51.453093,0.001095,0.507
3,0,0,3,4695978684,4698502266,35.778708,51.452902,0.001095,0.507
4,0,0,4,4695978684,4698502266,35.778521,51.451766,0.001095,0.507


In [ ]:
losses = []

# Find each trip (based on trip_id) and denoise it
for i in tqdm(range(20)):
    trip = test_dataset_noisy.loc[test_dataset_noisy["trip_id"] == i]
    denoised_trip = HMM(trip, pasdaran_streets)
    gt_trip = test_dataset_original.loc[test_dataset_original["trip_id"] == i]
    losses.append(mean_haversine_distance(denoised_trip, gt_trip))

print("Average loss:", np.mean(losses))

**How to submit**: You should submit your notebook and the `denoised_test_dataset.csv` file. Your codes are verified based on the "Mean haversine distance" value that your code gets and the `denoised_test_dataset.csv`.